In [1]:
from scipy.io import loadmat
import h5py
import re
import numpy as np
import validation

In [6]:
py_filepath = 'C:/Users/camil/Documents/int/inVibe/results/Riesling/LIP/2022-06-09_10-40-34/2/Riesling_LIP_2022-06-09_10-40-34_2.npy'
mat_filepath = "C:/Users/camil/Documents/int/data/openephys/2022-06-09_10-40-34/Record Node 102/experiment1/recording1/220609_TSCM_grid_Riesling.mat"

In [7]:
mat_res = validation.read_matlab_file(mat_filepath)
py_f = np.load(py_filepath,allow_pickle=True).item(0)
print(mat_res.keys())
print(py_f.keys())

dict_keys(['/data/NEURO/CodeNumbers', '/data/NEURO/CodeTimes', '/data/NEURO/Eyes/samples', '/data/NEURO/LFP/samples', '/data/NEURO/LFP/timestamps', '/data/NEURO/MUA', '/data/NEURO/Neuron'])
dict_keys(['times', 'block', 'clusters_id', 'clustersch', 'clustersgroup', 'clusterdepth', 'code_numbers', 'code_times', 'eyes_sample', 'lfp_sample', 'timestamps'])


#### Check neurons and mua times

In [8]:
start_idx = np.where(mat_res['/data/NEURO/CodeNumbers'].reshape(1,-1)[0]==9)[0]
end_idx = np.where(mat_res['/data/NEURO/CodeNumbers'].reshape(1,-1)[0]==18)[0]
start_timestamps = mat_res['/data/NEURO/CodeTimes'][0][start_idx]
end_timestamps = mat_res['/data/NEURO/CodeTimes'][0][start_idx[1:]]
end_timestamps = np.append(end_timestamps,[mat_res['/data/NEURO/LFP/timestamps'][0][-1]])

In [9]:
validation.check_spikes(mat_res, py_f, start_timestamps, end_timestamps)

Number of neurons do match
Number of mua do match
Number of spikes do match
Spike times do match
